In [10]:
%run ../Requirements.ipynb

In [11]:
conn = mysql.connector.connect(
    host='127.0.0.1',
    user='root',
    password='ronitgupta28',
    database='MusicFusion'
)
cursor = conn.cursor()

### Football

In [12]:
url = 'https://newsdata.io/api/1/news?apikey=pub_46893b2e912e5641cb98a854ddf2e7b4a0a7d&q=football&language=en&category=sports'

response = requests.get(url)
data = response.json()

In [13]:
if response.status_code == 200 and data['status'] == 'success':
    articles = data['results']
    df = pd.DataFrame(articles)
else:
    print(f"Error: {response.status_code}, {data['message']}")
    
df = df.drop(['keywords', 'creator', 'video_url', 'content', 'country', 'category', 'ai_tag', 'sentiment', 'sentiment_stats', 'ai_region', 'ai_org'], axis=1)
df = df.dropna()

In [14]:
df.to_parquet('/Users/ronitguptaaa/Documents/Music-Fusion-APP/Mart/SportsNews/Football')

In [22]:
columns = ", ".join([f"`{col}` VARCHAR(1024)" for col in df.columns])

drop_table_query = "DROP TABLE IF EXISTS FootballNews;"
cursor.execute(drop_table_query)

create_table_query = f"CREATE TABLE IF NOT EXISTS FootballNews ({columns});"
cursor.execute(create_table_query)

insert_query = f"INSERT INTO FootballNews ({', '.join(df.columns)}) VALUES ({', '.join(['%s'] * len(df.columns))})"
delete_query = f"DELETE FROM FootballNews WHERE link is NULL;"
for index, row in df.iterrows():
    values = tuple([None if pd.isna(val) else val for val in row])
    cursor.execute(insert_query, values)

cursor.execute(delete_query)
conn.commit()

### Cricket

In [33]:
url = "https://newsdata.io/api/1/news?apikey=pub_46893b2e912e5641cb98a854ddf2e7b4a0a7d&q=cricket&language=en&category=sports"

response = requests.get(url)
data = response.json()

In [44]:
if response.status_code == 200 and data['status'] == 'success':
    articles = data['results']
    df = pd.DataFrame(articles)
else:
    print(f"{response.status_code}, {data['results']['message']}")
    
df = df.drop(['keywords', 'creator', 'video_url', 'content', 'country', 'category', 'ai_tag', 'sentiment', 'sentiment_stats', 'ai_region', 'ai_org'], axis=1)
df = df.dropna()

In [46]:
df.to_parquet('/Users/ronitguptaaa/Documents/Music-Fusion-APP/Mart/SportsNews/Cricket')

In [51]:
columns = ", ".join([f"`{col}` VARCHAR(1024)" if col != 'description' else f"`{col}` TEXT" for col in df.columns])

drop_table_query = "DROP TABLE IF EXISTS CricketNews"
cursor.execute(drop_table_query)

create_table_query = f"CREATE TABLE IF NOT EXISTS CricketNews ({columns});"
cursor.execute(create_table_query)

insert_query = f"INSERT INTO CricketNews ({', '.join(df.columns)}) VALUES ({' ,'.join(['%s'] * len(df.columns))})"

for index, row in df.iterrows():
    values = tuple([None if pd.isna(val) else val for val in row])
    cursor.execute(insert_query, values)
    
conn.commit()

### Formula one